# 毎日家に帰ってきたらこれを一回走らせよう

In [36]:
from requests_oauthlib import OAuth1Session
import json
import settings

In [37]:
twitter = OAuth1Session(settings.CONSUMER_KEY, settings.CONSUMER_SECRET, settings.ACCESS_TOKEN, settings.ACCESS_TOKEN_SECRET)

In [38]:
import search_word

In [45]:
import sqlite3
conn = sqlite3.connect('bakayo_VS_chimon.db')
c = conn.cursor()

In [40]:
import pandas as pd

## ツイートの更新を取得する関数を定義

In [70]:
def update_tweet(tbl_name, sw):
    query = '''
    SELECT max(id) FROM {tbl}
    '''.format(tbl=tbl_name)
    maxid = c.execute(query)

    list_maxid = c.fetchall()
    max_id = list_maxid[0][0]

    df_tbl = pd.DataFrame(columns=['id', 'week','month','day','time','jisa','year'])

    if max_id == None:
        max_id = 0
    
    params = {}
    req = twitter.get("https://api.twitter.com/1.1/search/tweets.json?q={word}&max_id=811393552773095423&count=100".format(word=sw), params = params)
    tweets = json.loads(req.text)
    for tweet_statuses in tweets['statuses']:
        tweet_id = tweet_statuses['id']
        time = tweet_statuses['created_at'].split(' ')
        #if tweet_id > max_id:
        if tweet_id > 811013015856287743:
            print(tweet_statuses['id'], tweet_statuses['created_at'])
            insert_query = '''
            INSERT INTO {tbl} VALUES({tweet_id}, '{t1}', '{t2}', {t3}, '{t4}', '{t5}', {t6})
            '''.format(tbl=tbl_name,tweet_id=tweet_id,t1=time[0],t2=time[1],t3=time[2],t4=time[3],t5=time[4],t6=time[5])
            c.execute(insert_query)
    
            df_addition_row = pd.DataFrame([[tweet_id, time[0], time[1], time[2], time[3], time[4], time[5]]])
            df_addition_row.columns = ['id', 'week','month','day','time','jisa','year']
            df_tbl = pd.concat([df_tbl,df_addition_row],axis=0)
        
    conn.commit()
    
    return df_tbl

In [58]:
811552741046652929

811552741046652929

In [59]:
df_chimon = update_tweet('chimon_time', search_word.chimon)

811541845285040128 Wed Dec 21 12:00:28 +0000 2016


IntegrityError: UNIQUE constraint failed: chimon_time.id

In [ ]:
811934802911830016

In [71]:
df_bakayo = update_tweet('bakayo_time', search_word.bakayo)

811390054677348352 Wed Dec 21 01:57:18 +0000 2016


IntegrityError: UNIQUE constraint failed: bakayo_time.id

In [72]:
conn.close()